In [ ]:
from matplotlib import pyplot as plt
from omegaconf import OmegaConf
from rware.warehouse import Warehouse
from torch_geometric.utils import to_networkx
import networkx as nx

from diffusion_co_design.pretrain.rware.generate import (
    generate,
    WarehouseRandomGeneratorConfig,
)
from diffusion_co_design.pretrain.rware.transform import (
    storage_to_layout,
)
from diffusion_co_design.pretrain.rware.graph import generate_layout_graph
from diffusion_co_design.utils import omega_to_pydantic

config = omega_to_pydantic(
    OmegaConf.load(
        "../diffusion_co_design/bin/conf/scenario/rware_16_50_5_4_corners.yaml"
    ),
    WarehouseRandomGeneratorConfig,
)

out = generate(
    size=config.size,
    n_shelves=config.n_shelves,
    agent_idxs=config.agent_idxs,
    goal_idxs=config.goal_idxs,
    n_colors=config.n_colors,
)

layout = storage_to_layout(
    shelf_im=out[0],
    agent_idxs=config.agent_idxs,
    agent_colors=config.agent_colors,
    goal_idxs=config.goal_idxs,
    goal_colors=config.goal_colors,
)

data = generate_layout_graph(layout)

g = to_networkx(data, to_undirected=True)
nx.draw(g)